In [1]:
# !git clone https://github.com/WongKinYiu/yolov7.git /home/jovyan/yolov7

# mkdir weights
# curl https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt > weights/yolov7_training.pt

In [3]:
%%bash
# !python -m ipykernel install --user --name yolov7 --display-name yolov7
# !{sys.executable} -m pip install -r requirements.txt --quiet
python -m pip install kfp==1.8.22 --user --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
import sys
import os

import kfp
from kfp import compiler, dsl, components
from kfp.components import InputPath, OutputPath
from typing import NamedTuple
from kfp.dsl import component
from typing import Dict, List

In [18]:
kfp.__version__

'1.8.22'

In [82]:
# @component(
#     packages_to_install=['google-cloud-storage', 'numpy', 'chevron', 'tarfile']
# )
def prepare_labelstudio_data_for_yolo(
        project: str,
        labels: list,
        namespace: str,
        domain: str,
        config_template_url: str,
        transfer_weights_url: str,
        hyp_file_url: str,
        train_frac: float,
        validate_frac: float,
        workspace: OutputPath(str)
) -> NamedTuple('YOLOArgs',
                [('data_file', str),
                 ('config_file', str),
                 ('weights_file', str),
                 ('hyp_file', str),
                 ('names_file', str)]):
    '''
    Prepares Labelstudio Data for YOLO training
    Example weights_url: https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt
    '''

    print(f'Prepares Labelstudio Data for training of YOLOv7')

    import chevron
    import os
    import teknoir_labelstudio_sdk
    import zipfile
    import json
    from random import shuffle
    from pathlib import Path
    from google.cloud import storage
    from urllib.parse import urlparse
    from collections import namedtuple
    import tarfile

    def create_directory(directory_name, basedir=None):
        if basedir is None:
            pth = Path(directory_name)
        else:
            pth = Path(os.path.join(basedir, directory_name))
        pth.mkdir(parents=True, exist_ok=True)
        return pth.as_posix()

    # Directory structure
    basedir = create_directory(workspace)  # /workspace
    configdir = create_directory('config', basedir)  # /workspace/model
    datadir = create_directory('dataset', basedir)  # /workspace/dataset
    imagesdir = create_directory('images', datadir)  # /workspace/dataset/images
    trainimagesdir = create_directory('train', imagesdir)  # /workspace/dataset/images/train
    valimagesdir = create_directory('val', imagesdir)  # /workspace/dataset/images/val
    testimagesdir = create_directory('test', imagesdir)  # /workspace/dataset/images/test
    labelsdir = create_directory('labels', datadir)  # /workspace/dataset/labels
    trainlabelsdir = create_directory('train', labelsdir)  # /workspace/dataset/labels/train
    vallabelsdir = create_directory('val', labelsdir)  # /workspace/dataset/labels/val
    testlabelsdir = create_directory('test', labelsdir)  # /workspace/dataset/labels/test

    # Connect to the Label Studio API and check the connection
    ls = teknoir_labelstudio_sdk.Client(
        extra_headers=teknoir_labelstudio_sdk.jwt_header('kubeflow.training@teknoir.ai'))
    print(ls.check_connection())

    projects = ls.list_projects(title=project)
    p = projects[0]
    print(p.parsed_label_config)

    yolo_annotations_file_zip = p.export_tasks(export_type="YOLO", download_resources=True,
                                                     export_location=f'{workspace}/{project}.zip')

    tasks_json_file = p.export_tasks(export_type="JSON", download_resources=True,
                                           export_location=f'{workspace}/{project}.json')

    with open(tasks_json_file) as f:
        tasks = json.load(f)

    if len(tasks) == 0:
        print(f'NO TASKS FOUND FOR {project}')
        exit(0)

    ind_list = list(range(len(tasks)))
    shuffle(ind_list)
    test_frac = 1.0 - train_frac - validate_frac
    train_set = ind_list[:int((len(ind_list) + 1) * train_frac)]
    test_set = ind_list[int((len(ind_list) + 1) * train_frac):int((len(ind_list) + 1) * (train_frac + test_frac))]
    val_set = ind_list[int((len(ind_list) + 1) * (train_frac + test_frac)):]

    print(f'Number of tasks: {len(tasks)}')
    print(f'Train set size: {len(train_set)}')
    print(f'Validate set size: {len(test_set)}')
    print(f'Test set size: {len(val_set)}')

    bucket_name = f'{namespace}.{domain}'
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    labelstudio_upload_location = 'label-studio/data'

    with zipfile.ZipFile(yolo_annotations_file_zip, 'r') as zip_ref:
        zip_ref.extractall(f'{workspace}/{project}/yolo')

    yolo_labelsdir = f'{workspace}/{project}/yolo/labels'

    # DATA SETS
    def write_dataset(tasks, set, imagesdir, labelsdir):
        for i in set:
            task = tasks[i]
            if task['data']['image']:
                image_path = task['data']['image']
                image_name = Path(image_path).name
                source_image = f'{labelstudio_upload_location}/{image_path}'
                dest_image = f'{imagesdir}/{image_name}'
                current_blob = bucket.get_blob(source_image)
                current_blob.download_to_filename(dest_image)
                print(f"Image {image_name} was downloaded to {dest_image}.")

                source_labels = f'{yolo_labelsdir}/{Path(image_name).stem}.txt'
                dest_labels = f'{labelsdir}/{Path(image_name).stem}.txt'
                os.rename(source_labels, dest_labels)
                print(f"Moved labels from {source_labels} to {dest_labels}.")

    # Write training dataset
    write_dataset(tasks, train_set, trainimagesdir, trainlabelsdir)

    # Write validation dataset
    write_dataset(tasks, val_set, valimagesdir, vallabelsdir)

    # Write test dataset
    write_dataset(tasks, test_set, testimagesdir, testlabelsdir)

    # CONFIG
    print(f"Writing CONFIG file")
    data_file = os.path.join(configdir, f'data.yaml')
    print(f"- DST: {data_file}")

    num_classes = len(labels)

    names_file = os.path.join(datadir, "object.names")
    train_data_file = os.path.join(datadir, "train.txt")
    val_data_file = os.path.join(datadir, "val.txt")
    test_data_file = os.path.join(datadir, "test.txt")

    with open(data_file, 'w') as out:
        out.write(f'train: {train_data_file}\n')
        out.write(f'val: {val_data_file}\n')
        out.write(f'test: {test_data_file}\n')
        out.write(f'nc: {num_classes}\n')
        out.write(f'names: [{", ".join(labels)}]')

    with open(names_file, 'w') as out:
        for l in labels:
            out.write(l + '\n')

    # /workspace/dataset/train.txt
    with open(train_data_file, 'w') as out:
        for f in os.listdir(trainimagesdir):
            out.write(f'{os.path.join(trainimagesdir, f)}\n')

    # /workspace/dataset/val.txt
    with open(val_data_file, 'w') as out:
        for f in os.listdir(valimagesdir):
            out.write(f'{os.path.join(valimagesdir, f)}\n')

    # /workspace/dataset/test.txt
    with open(test_data_file, 'w') as out:
        for f in os.listdir(testimagesdir):
            out.write(f'{os.path.join(testimagesdir, f)}\n')

    # CONFIG (from GCS)
    print(f"Writing YOLOv7 CONFIG file")
    print(f"- SRC: {config_template_url}")
    custom_config_file = os.path.join(configdir, "config.yaml")
    print(f"- DST: {custom_config_file}")
    parsed_url = urlparse(config_template_url)
    blob = bucket.blob(parsed_url.path[1:])
    with open(custom_config_file, 'w') as fout:
        parsed_config = chevron.render(blob.download_as_string().decode(),
                                       {"num_classes": num_classes})
        # to configure anything else, create a copy, modify, and reference.
        # full parameterization would likely be more confusing than helpful
        fout.write(parsed_config)
        print(parsed_config)

    # HYP FILE (from GCS)
    print("Copying HYP file")
    print(f"- SRC: {hyp_file_url}")
    hyp_file = os.path.join(configdir, "hyp.yaml")
    print(f"- DST: {hyp_file}")
    parsed_url = urlparse(hyp_file_url)
    blob = bucket.blob(parsed_url.path[1:])
    blob.download_to_filename(hyp_file)

    # PRETRAINED WEIGHTS (from GCS)
    transfer_weights_file = ""
    if transfer_weights_url:
        print(f"Copying weights for transfer learning")
        print(f"- SRC: {transfer_weights_url}")
        transfer_weights_file = os.path.join(configdir, 'transfer_weights.pt')
        print(f"- DST: {transfer_weights_file}")
        parsed_url = urlparse(transfer_weights_url)
        blob = bucket.blob(parsed_url.path[1:])
        blob.download_to_filename(transfer_weights_file)

    output = namedtuple('YOLOArgs', ['data_file', 'config_file', 'transfer_weights_file', 'hyp_file', 'names_file'])
    return output(data_file, custom_config_file, transfer_weights_file, hyp_file, names_file)


prepare_labelstudio_data_for_yolo_op = components.create_component_from_func(
    prepare_labelstudio_data_for_yolo,
    base_image='python:3.8',
    packages_to_install=['google-cloud-storage', 'numpy', 'chevron', 'git+https://github.com/teknoir/teknoir-labelstudio-sdk.git#egg=teknoir-labelstudio-sdk'],
    output_component_file='prepare_labelstudio_data_for_yolo.yaml')

In [83]:
def train_op(model_name: str,
             workspace_path: InputPath(str),
             weights: str,
             data: str,
             config: str,
             hyp: str,
             batch_size: int,
             img_size_train: int,
             img_size_test: int,
             epochs: int):
    return dsl.ContainerOp(
        name='Train YOLOv7',
        image='gcr.io/teknoir/yolov7-training:main-amd64',  # nvidia pytorch base image is VERY large
        command=['sh', '-c'],
        arguments=[f'''echo "WEIGHTS: {weights}" &&
            echo "DATA: {data}" &&
            echo "CONFIG: {config}" &&
            echo "HYP: {hyp}" &&
            python3 train.py --workers 0 --device 0 --batch-size {batch_size} --data {data} --img {img_size_train} {img_size_test} --cfg {config} --name {model_name} --weights {weights} --hyp {hyp} --epochs {epochs} --project=/workspace --name=model --exist-ok'''],
        container_kwargs={'env': [V1EnvVar('MODEL_NAME', model_name)]},
        artifact_argument_paths=[workspace_path],
        file_outputs={
            'best_model': '/workspace/model/weights/best.pt',
            'last_model': '/workspace/model/weights/last.pt',
            'names': '/workspace/dataset/object.names',
            'data': '/workspace/config/data.yaml',
            'config': '/workspace/config/config.yaml',
            'hyp': '/workspace/config/hyp.yaml',
            'F1_curve_image': '/workspace/model/F1_curve.png',
            'P_curve_image': '/workspace/model/P_curve.png',
            'PR_curve_image': '/workspace/model/PR_curve.png',
            'R_curve_image': '/workspace/model/R_curve.png',
            'results_txt': '/workspace/model/results.txt',
            'results_image': '/workspace/model/results.png',
            'confusion_matrix_image': '/workspace/model/confusion_matrix.png',
        }
    )

In [84]:
# @component(
#     packages_to_install=['google-cloud-storage', 'six']
# )
def save_yolo_model(model_name: str,
                    namespace: str,
                    domain: str,
                    description: str,
                    best_model: InputPath(str),
                    last_model: InputPath(str),
                    names: InputPath(str),
                    data: InputPath(str),
                    config: InputPath(str),
                    hyp: InputPath(str),
                    f1_curve: InputPath(str),
                    p_curve: InputPath(str),
                    pr_curve: InputPath(str),
                    r_curve: InputPath(str),
                    results_txt: InputPath(str),
                    results_image: InputPath(str),
                    confusion_matrix_image: InputPath(str)):
    # -> NamedTuple('YOLOModel', [('outputs', dict)]):
    from google.cloud import storage
    import os
    from collections import namedtuple
    # from pathlib import Path

    def upload_blob(bucket_name, source_file_name, destination_blob_name):
        """Uploads a file to the bucket."""
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(source_file_name)
        print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))

    bucket_name = f"{namespace}.{domain}"
    destination_prefix = os.path.join('models', model_name)

    # 'yolov7-tiny.weights', 'yolov7-tiny.cfg', 'object.names', 'confusion_matrix.png',
    # 'F1_curve.png', 'P_curve.png', 'PR_curve.png', 'R_curve.png', 'results.png', 
    # 'hyp.yaml', 'opt.yaml', ...

    #Upload best weights file (runs/train/<model name>/weights/best.pt)
    upload_blob(bucket_name, best_model, os.path.join(destination_prefix, "best_weights.pt"))

    #Upload last weights file as well
    upload_blob(bucket_name, last_model, os.path.join(destination_prefix, "last_weights.pt"))

    #Upload object names - eventually deprecate in favor of data.yaml
    upload_blob(bucket_name, names, os.path.join(destination_prefix, "object.names"))

    #Upload data.yaml
    upload_blob(bucket_name, data, os.path.join(destination_prefix, "training_data.yaml"))

    #Upload config.yaml
    upload_blob(bucket_name, config, os.path.join(destination_prefix, "training_config.yaml"))

    #Upload hyp.yaml
    upload_blob(bucket_name, hyp, os.path.join(destination_prefix, "training_hyp.yaml"))
    
    #Upload
    upload_blob(bucket_name, f1_curve, os.path.join(destination_prefix, "F1_curve.png"))
    upload_blob(bucket_name, p_curve, os.path.join(destination_prefix, "P_curve.png"))
    upload_blob(bucket_name, pr_curve, os.path.join(destination_prefix, "PR_curve.png"))
    upload_blob(bucket_name, r_curve, os.path.join(destination_prefix, "R_curve.png"))
    upload_blob(bucket_name, results_txt, os.path.join(destination_prefix, "results.txt"))
    upload_blob(bucket_name, results_image, os.path.join(destination_prefix, "results.png"))
    upload_blob(bucket_name, confusion_matrix_image, os.path.join(destination_prefix, "confusion_matrix.png"))

    catalog_info_t = f'''
---
apiVersion: backstage.io/v1alpha1
kind: Component
metadata:
  title: {model_name}
  name: {model_name}
  namespace: {namespace}
  description: {description}
  annotations:
    'teknoir.org/model-registry/f1_curve': {os.path.join(destination_prefix, "F1_curve.png")}
    'teknoir.org/model-registry/p_curve': {os.path.join(destination_prefix, "P_curve.png")}
    'teknoir.org/model-registry/pr_curve': {os.path.join(destination_prefix, "PR_curve.png")}
    'teknoir.org/model-registry/r_curve': {os.path.join(destination_prefix, "R_curve.png")}
    'teknoir.org/model-registry/results_txt': {os.path.join(destination_prefix, "results.txt")}
    'teknoir.org/model-registry/results': {os.path.join(destination_prefix, "results.png")}
    'teknoir.org/model-registry/confusion_matrix': {os.path.join(destination_prefix, "confusion_matrix.png")}
spec:
  type: model
  owner: group:{namespace}/{namespace}
  lifecycle: experimental
  system: system:{namespace}/{namespace}
'''
    with open("catalog-info.yaml", 'w') as f:
        f.write(catalog_info_t)
    upload_blob(bucket_name, "catalog-info.yaml", os.path.join(destination_prefix, "catalog-info.yaml"))


save_yolo_model_op = components.create_component_from_func(
    save_yolo_model,
    base_image='python:3.8',
    packages_to_install=['google-cloud-storage', 'six'],
    output_component_file='save_yolo_model.yaml')

In [85]:
# import os
# import kubernetes as k8s
from kubernetes.client.models import V1EnvVar, V1ContainerPort

pipeline_name = 'Train YOLOv7 with Labelstudio and 1 T4 GPU'
pipeline_description = 'A pipeline to train YOLOv7 on a custom data set from Teknoir Labelstudio. Transfer learn from "transfer_weights."'


@dsl.pipeline(name=pipeline_name, description=pipeline_description)
def train_yolov7(model_name: str,
                 labelstudio_project: str,
                 labels: list,
                 namespace: str,
                 domain: str,
                 config_template_url: str = 'gs://teknoir-ai.teknoir.cloud/yolov7/cfg/yolov7-tiny-config-template.yaml',
                 transfer_weights_url: str = 'gs://teknoir-ai.teknoir.cloud/yolov7/weights/yolov7_training.pt',
                 hyp_file_url: str = 'gs://teknoir-ai.teknoir.cloud/yolov7/hyp/hyp.scratch.tiny.yaml',
                 train_frac: float = 0.7,
                 # fraction of samples to use for training (test_frac = 1 - train_frac - validate_frac)
                 validate_frac: float = 0.2,
                 # fraction of samples to use for validation (test_frac = 1 - train_frac - validate_frac)
                 batch_size: int = 16,
                 img_size_train: int = 416,
                 img_size_test: int = 416,
                 epochs: int = 300,
                 ):
    """Train YOLOv7 with Labelstudio"""

    gpu_instance_type = "nvidia-tesla-t4"  # https://cloud.google.com/compute/docs/gpus

    prepare = prepare_labelstudio_data_for_yolo_op(
        project=labelstudio_project,
        labels=labels,
        namespace=namespace,
        domain=domain,
        config_template_url=config_template_url,
        transfer_weights_url=transfer_weights_url,
        hyp_file_url=hyp_file_url,
        train_frac=train_frac,
        validate_frac=validate_frac
    )
    prepare.execution_options.caching_strategy.max_cache_staleness = "P0D"
    prepare.add_pod_annotation('proxy.istio.io/config', '{"holdApplicationUntilProxyStarts": true }')
    prepare.add_pod_annotation('sidecar.istio.io/inject', 'true')

    
    train = train_op(
        workspace_path=kfp.dsl.InputArgumentPath(argument=prepare.outputs['workspace'], path='/tmp/outputs/workspace/data'),
        model_name=model_name,
        weights=prepare.outputs['weights_file'],
        data=prepare.outputs['data_file'],
        config=prepare.outputs['config_file'],
        hyp=prepare.outputs['hyp_file'],
        batch_size=batch_size,
        img_size_train=img_size_train,
        img_size_test=img_size_test,
        epochs=epochs)
    train.after(prepare) \
        .add_port(V1ContainerPort(container_port=8099)) \
        .add_port(V1ContainerPort(container_port=8079)) \
        .set_gpu_limit(1) \
        .add_node_selector_constraint('cloud.google.com/gke-accelerator', gpu_instance_type) \
        .execution_options.caching_strategy.max_cache_staleness = "P0D"


    save_model = save_yolo_model_op(
        model_name=model_name,
        namespace=namespace,
        domain=domain,
        description=pipeline_description,
        best_model=train.outputs['best_model'],
        last_model=train.outputs['last_model'],
        names=train.outputs['names'],
        data=train.outputs['data'],
        config=train.outputs['config'],
        hyp=train.outputs['hyp'],
        f1_curve=train.outputs['F1_curve_image'],
        p_curve=train.outputs['P_curve_image'],
        pr_curve=train.outputs['PR_curve_image'],
        r_curve=train.outputs['R_curve_image'],
        results_txt=train.outputs['results_txt'],
        results_image=train.outputs['results_image'],
        confusion_matrix_image=train.outputs['confusion_matrix_image']
    ).after(train)
    save_model.execution_options.caching_strategy.max_cache_staleness = "P0D"

In [86]:
client = kfp.Client(namespace='teknoir')
# 5h timeout
pipeline_conf = kfp.dsl.PipelineConf().set_timeout(3600 * 5).set_image_pull_policy(policy="Always")
workflow = kfp.compiler.Compiler().compile(pipeline_func=train_yolov7,
                                           package_path="train_yolov7.yaml",
                                           pipeline_conf=pipeline_conf)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


In [87]:
import uuid
import json

pipeline_version_file = pipeline_file = 'train_yolov7.yaml'

client = kfp.Client(namespace='teknoir')
# 5h timeout
pipeline_conf = kfp.dsl.PipelineConf().set_timeout(3600 * 5).set_image_pull_policy(policy="Always")
workflow = kfp.compiler.Compiler().compile(pipeline_func=train_yolov7,
                                           package_path=pipeline_file,
                                           pipeline_conf=pipeline_conf)

filter = json.dumps({'predicates': [{'key': 'name',
                                     'op': 1,
                                     'string_value': pipeline_name}]})
pipelines = client.pipelines.list_pipelines(filter=filter)

if not pipelines.pipelines:
    pipeline = client.pipeline_uploads.upload_pipeline(pipeline_file,
                                                       name=pipeline_name,
                                                       description=pipeline_description)
else:
    pipeline_version_name = pipeline_name + f' - {str(uuid.uuid4())[:6]}'
    pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_version_file,
                                                                       name=pipeline_version_name,
                                                                       pipelineid=pipelines.pipelines[0].id)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').
